In [7]:
#STEP 1: IMPORTS
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.models import mobilenet_v2, MobileNet_V2_Weights
from tqdm import tqdm
import time
#STEP 2: Fix Transforms

IMAGE_SIZE = 224

train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
])
import os
import shutil
import random

source_dir = "garbage_classification"       # your only folder
dest_dir = "dataset"         # new split folder
train_ratio = 0.8            # 80% training, 20% testing

# Create output folders
train_dir = os.path.join(dest_dir, "train")
test_dir  = os.path.join(dest_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir,  exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)

    if not os.path.isdir(class_path):
        continue
    
    images = os.listdir(class_path)
    random.shuffle(images)

    split_idx = int(len(images) * train_ratio)
    train_files = images[:split_idx]
    test_files  = images[split_idx:]

    # Create class folders inside train/test
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name),  exist_ok=True)

    # Move files to train folder
    for img in train_files:
        shutil.copy(
            os.path.join(class_path, img),
            os.path.join(train_dir, class_name, img)
        )

    # Move files to test folder
    for img in test_files:
        shutil.copy(
            os.path.join(class_path, img),
            os.path.join(test_dir, class_name, img)
        )

    print(f"Class '{class_name}' → {len(train_files)} train, {len(test_files)} test")

print("Dataset split completed successfully!")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)
train_data = datasets.ImageFolder("dataset/train", transform=train_transforms)
test_data  = datasets.ImageFolder("dataset/test", transform=test_transforms)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4)
test_loader  = DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4)

class_names = train_data.classes

#STEP 3: Model Factory
from torchvision.models import (
    mobilenet_v2, MobileNet_V2_Weights,
    resnet50, ResNet50_Weights,
    efficientnet_b0, EfficientNet_B0_Weights
)

def get_model(model_name, num_classes):
    if model_name == "mobilenetv2":
        model = mobilenet_v2(weights=MobileNet_V2_Weights.DEFAULT)
        for p in model.features.parameters():
            p.requires_grad = False
        model.classifier[1] = nn.Linear(
            model.classifier[1].in_features, num_classes
        )

    elif model_name == "resnet50":
        model = resnet50(weights=ResNet50_Weights.DEFAULT)
        for p in model.parameters():
            p.requires_grad = False
        model.fc = nn.Linear(model.fc.in_features, num_classes)

    elif model_name == "efficientnetb0":
        model = efficientnet_b0(weights=EfficientNet_B0_Weights.DEFAULT)
        for p in model.features.parameters():
            p.requires_grad = False
        model.classifier[1] = nn.Linear(
            model.classifier[1].in_features, num_classes
        )

    return model.to(device)

#STEP 4: Train + Evaluate Function
def train_and_evaluate(model, train_loader, test_loader, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=0.001
    )

    start_time = time.time()

    # Training
    for epoch in range(epochs):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    train_time = time.time() - start_time

    # Evaluation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    test_acc = correct / total

    return test_acc, train_time

#STEP 5: Run Model Comparison

models_to_compare = ["mobilenetv2", "resnet50", "efficientnetb0"]
results = []

for model_name in models_to_compare:
    print(f"\n🚀 Training {model_name.upper()}")
    model = get_model(model_name, len(class_names))

    acc, ttime = train_and_evaluate(
        model,
        train_loader,
        test_loader,
        epochs=5
    )

    params = sum(p.numel() for p in model.parameters())

    results.append({
        "Model": model_name,
        "Test Accuracy": round(acc, 4),
        "Train Time (s)": round(ttime, 2),
        "Parameters": params
    })

#STEP 6: Print Results Table
import pandas as pd

df_results = pd.DataFrame(results)
print(df_results)

#STEP 7: Automatically Pick Best Model
best_model = df_results.sort_values(
    by=["Test Accuracy", "Train Time (s)"],
    ascending=[False, True]
).iloc[0]

print("\n🏆 BEST MODEL:")
print(best_model)


Class 'battery' → 756 train, 189 test
Class 'biological' → 788 train, 197 test
Class 'brown-glass' → 485 train, 122 test
Class 'cardboard' → 712 train, 179 test
Class 'clothes' → 4260 train, 1065 test
Class 'green-glass' → 503 train, 126 test
Class 'metal' → 615 train, 154 test
Class 'paper' → 840 train, 210 test
Class 'plastic' → 692 train, 173 test
Class 'shoes' → 1581 train, 396 test
Class 'trash' → 557 train, 140 test
Class 'white-glass' → 620 train, 155 test
Dataset split completed successfully!
Running on: cpu

🚀 Training MOBILENETV2

🚀 Training RESNET50
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\Murugan/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1032)>

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context